In [1]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import Multi30k
from dataloader import *
from utils import *
from torch.optim import Adam

In [4]:
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float("inf")
emb_dim = 256
enc_hid_dim = dec_hid_dim = 512
n_layers = 10
dropout = 0.25
batch_size = 128
kernel_size = 3
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)

In [3]:
train_dataset, val_dataset, test_dataset = Multi30k(root="data")
train_dataset, val_dataset, test_dataset = (
    to_map_style_dataset(train_dataset),
    to_map_style_dataset(val_dataset),
    to_map_style_dataset(test_dataset),
)
train_dataloader, val_dataloader, test_dataloader, etc = get_dataloader_and_etc(
    train_dataset,
    val_dataset,
    test_dataset,
    batch_size,
)
_, _, vocab_de, vocab_en = etc
input_dim = len(vocab_de)
output_dim = len(vocab_en)

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout):
        super().__init__()
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        self.droput = nn.Dropout(dropout)

    def forward(self, query, key, value, mask = None):
        batch_size = query.shape[0]

        Q = self.fc_q()
        


In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout):
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = None

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self,
        input_dim,
        hid_dim,
        n_layers,
        n_heads,
        pf_dim,
        dropout,
        max_length=100,
    ):
        super().__init__()
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        